In [ ]:
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

## `SciPy.integrate`: Numerická integrace a řešení diferenciálních rovnic

### Vyčíslení určitého integrálu
Numerical evaluation of a function of the type
Často potřebujeme numericky vyčíslit určitý integrál, tj.

$\displaystyle \int_a^b f(x) {\rm d}x$

Numerické integraci se často říká kvadratura, anglicky *quadrature*. Podle toho se jmenují o funkce v modulu `scipy.integrate`, např. `quad`, `dblquad`, `tplquad` nebo obecné `nquad`.

In [ ]:
import scipy.integrate


Zkusíme spočítat jednodychý integrál:

$\displaystyle \int_0^1 x {\rm d}x$

In [ ]:
val, abserr = sp.integrate.quad(lambda x: x, 0, 1)
print(f"výsledek = {val:g} ± {abserr:.2g}")

Můžeme dokonce pracovat s nekonečnými mezemi.

$$\displaystyle \int_{-\infty}^\infty e^{-x^2} {\rm d}x$$

In [ ]:
val, abserr = sp.integrate.quad(lambda x: np.exp(-x ** 2), -np.Inf, np.Inf)
print(f"výsledek = {val:g} ± {abserr:.2g}")
print(f"rozdíl od přesné hodnoty (√π) = {val - np.sqrt(np.pi):g}")


### Obyčejné diferenciální rovnice (ODR)

`scipy.integrate` (ano, řešení ODR je v tomto modulu, protože řešením ODR je určitý integrál) obsahuje `odeint`, které je jednodušší, a objektové rozhraní `ode`, které umožňuje větší kontrolu.


ODR (nebo jejich soustava) je často zadaná jako

$y' = f(y, t)$

s počátečními podmínkami

$y(t=0) = y_0$

`odeint` pak lze použít jednoduše:

    y_t = odeint(f, y_0, t)

kde `t` je předem zadané pole, ve kterých požadujeme řešení.

#### Příklad 1: jednoduché kyvadlo

Rovnice jednoduchého fyzikálního kyvadla je

Zrychlení kyvadla $\theta ''$ závisí na pozici $\theta$ a gravitační konstanty $g$ a délky kyvadla $L$.

$\displaystyle {\theta ''} = - \frac{g}{L}\theta$

Řešení je známé, použijeme jej pro kontrolu:

$\displaystyle {\theta} = \theta_0 \cos\left( \sqrt{\frac{g}{L}} t \right) $

Zkusme si tuto rovnici vyřešit numericky. Jelikož potřebujeme rovnice prvního řádu, definujeme vektor $x = \left(\theta , \theta ' \right)$, takže

$\displaystyle {x_1 '} = x_2$

$\displaystyle {x_2 '} = - \frac{g}{L}\theta $


In [ ]:
from scipy.constants import g

L = 0.5
m = 0.1

def dx_pendulum(x, t):
    """
    The right-hand side of the pendulum ODE
    """
    theta, dtheta = x[0], x[1]
    
    d_theta_dt = dtheta
    d_dtheta_dt = - g / L * theta
    
    return d_theta_dt, d_dtheta_dt

In [ ]:
# počáteční stav = pozice a rychlost
x0 = [np.pi / 8, 0]
# časy pro řešení
t = np.linspace(0, 10, 250)
# a konečně řešení
x = sp.integrate.odeint(dx_pendulum, x0, t)

In [ ]:
# analytické řešení
x_anal = x0[0] * np.cos(np.sqrt(g / L) * t)

In [ ]:
plt.plot(t, x[:, 0], 'r', label=r"$\theta$")
plt.plot(t, x_anal, 'k--', label=u"přesné řešení")
plt.legend()
plt.xlabel("Čas [s]")
plt.ylabel("Poloha")